# Inspector tutorial

#### What is inspector ?  
   Vai_q_pytorch provides a function called inspector to help users diagnose neural network (NN) models under different device architectures. The inspector can predict target device assignments based on hardware constraints.The generated inspection report can be used to guide  users to modify or optimize the NN model, greatly reducing the difficulty and time of deployment. It is recommended to inspect float models before quantization.

In [3]:
# Import inspector API
# 
# Note:
# You can ignore warning message related with XIR. 
# The inspector relies on 'vai_utf' package. In conda env vitis-ai-pytorch in Vitis-AI docker, vai_utf is ready. But if vai_q_pytorch is installed by source code, it needs to install vai_utf in advance.
from pytorch_nndct.apis import Inspector


[VAIQ_NOTE]: Loading NNDCT kernels...


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvBlock(nn.Module):
    """
    Convolution block:
        - 1d conv
        - layer norm by embedding axis
        - activation
        - dropout
        - Max pooling
    """

    def __init__(self, in_channels, out_channels, kernel_size,
                 stride=1, dilation=1, p_conv_drop=0.1):
        super(ConvBlock, self).__init__()

        # use it instead stride.

        self.conv1d = nn.Conv2d(in_channels, out_channels,
                                kernel_size=kernel_size,
                                bias=False,
                                padding='same')

        self.norm = nn.LayerNorm(out_channels)
        self.activation = nn.GELU()
        self.drop = nn.Dropout(p=p_conv_drop)

        self.downsample = nn.MaxPool2d(kernel_size=stride, stride=stride)

        self.stride = stride
        self.in_channels = in_channels
        self.out_channels = out_channels

    def forward(self, x):
        x = torch.unsqueeze(x, -1)
        x = self.conv1d(x)
        x = torch.squeeze(x, -1)

        # norm by last axis.
        x = torch.transpose(x, -2, -1)
        x = self.norm(x)
        x = torch.transpose(x, -2, -1)

        x = self.activation(x)

        x = self.drop(x)

        x = torch.unsqueeze(x, -1)
        x = self.downsample(x)
        x = torch.squeeze(x, -1)

        return x


class UpConvBlock(nn.Module):
    """
    Decoder convolution block
    """

    def __init__(self, scale, **args):
        super(UpConvBlock, self).__init__()
        self.conv_block = ConvBlock(**args)
        self.upsample = nn.Upsample(scale_factor=scale, mode='linear', align_corners=False)

    def forward(self, x):
        x = self.conv_block(x)
        x = self.upsample(x)
        return x


class AutoEncoder1D(nn.Module):
    """
    This is the final Encoder-Decoder model with skip connections
    """

    def __init__(self,
                 n_electrodes=30,  # Number of channels
                 n_freqs=16,  # Number of wavelets
                 n_channels_out=21,  # Number of fingers
                 channels=[8, 16, 32, 32],  # Number of features on each encoder layer
                 kernel_sizes=[3, 3, 3],
                 strides=[4, 4, 4],
                 dilation=[1, 1, 1]
                 ):

        super(AutoEncoder1D, self).__init__()

        self.n_electrodes = n_electrodes
        self.n_freqs = n_freqs
        self.n_inp_features = n_freqs * n_electrodes
        self.n_channels_out = n_channels_out

        self.model_depth = len(channels) - 1
        self.spatial_reduce = ConvBlock(self.n_inp_features, channels[0], kernel_size=3)  # Dimensionality reduction

        # Encoder part
        self.downsample_blocks = nn.ModuleList([ConvBlock(channels[i],
                                                          channels[i + 1],
                                                          (kernel_sizes[i], 1),
                                                          stride=(strides[i], 1),
                                                          dilation=(dilation[i], 1)) for i in range(self.model_depth)])

        channels = [ch for ch in channels[:-1]] + channels[-1:]  # channels

        # Decoder part
        self.upsample_blocks = nn.ModuleList([UpConvBlock(scale=strides[i],
                                                          in_channels=channels[i + 1] if i == self.model_depth - 1 else
                                                          channels[i + 1] * 2,
                                                          out_channels=channels[i],
                                                          kernel_size=(kernel_sizes[i], 1)) for i in
                                              range(self.model_depth - 1, -1, -1)])

        self.conv1x1_one = nn.Conv2d(channels[0] * 2, self.n_channels_out, kernel_size=(1,1),
                                     padding='same')  # final 1x1 conv

    def forward(self, x):

        batch, elec, n_freq, time = x.shape
        x = x.reshape(batch, -1, time)  # flatten the input
        x = self.spatial_reduce(x)

        skip_connection = []

        for i in range(self.model_depth):
            skip_connection.append(x)
            x = self.downsample_blocks[i](x)

        for i in range(self.model_depth):
            x = self.upsample_blocks[i](x)
            x = torch.cat((x, skip_connection[-1 - i]),  # skip connections
                          dim=1)

        x = torch.unsqueeze(x, -1)
        x = self.conv1x1_one(x)
        x = torch.squeeze(x, -1)
        return x


In [5]:
# Define a toy neural network
class ToyModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.upsample = torch.nn.Upsample(scale_factor=2, mode='bilinear')
        self.conv = torch.nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        self.relu = torch.nn.ReLU()
        self.adaptive_avg_pool2d = torch.nn.AdaptiveAvgPool2d(output_size=2)
        
    def forward(self, x):
        x = self.upsample(x)
        x = self.conv(x)
        x = self.relu(x)
        x = self.adaptive_avg_pool2d(x)
        x = x.reshape(x.size(0), -1)
        return x

In [6]:
# Specify a target name or fingerprint you want to deploy on
target = "DPUCZDX8G_ISA1_B4096"
# Initialize inspector with target
inspector = Inspector(target)


[VAIQ_NOTE]: Inspector is on.


In [7]:
# Model hyperparameters
hp_autoencoder = dict(
    channels=[32, 32, 64, 64, 128, 128],
    kernel_sizes=[7, 7, 5, 5, 5],
    strides=[2, 2, 2, 2, 2],
    dilation=[1, 1, 1, 1, 1],
    n_electrodes=62,
    n_freqs=40,
    n_channels_out=5
)

In [8]:
# Start to inspect the float model
# Note: visualization of inspection results relies on the dot engine.If you don't install dot successfully, set 'image_format = None' when inspecting.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoEncoder1D(**hp_autoencoder)
dummy_input = torch.randn(4, 62, 40, 256)
inspector.inspect(model, (dummy_input,), device=device, output_dir="inspect", image_format="png") 


[VAIQ_NOTE]: =>Start to inspect model...

[VAIQ_NOTE]: =>Quant Module is in 'cpu'.

[VAIQ_NOTE]: =>Parsing AutoEncoder1D...

[VAIQ_NOTE]: Start to trace and freeze model...

[VAIQ_NOTE]: The input model nndct_st_AutoEncoder1D_ed is torch.nn.Module.

[VAIQ_NOTE]: Finish tracing.

[VAIQ_NOTE]: Processing ops...


██████████████████████████████████████████████████| 139/139 [00:00<00:00, 1126.20it/s, OpInfo: name = return_0, type = R


[VAIQ_WARN][QUANTIZER_TORCH_FLOAT_OP]: The quantizer recognize new op `aten::upsample_linear1d` as a float operator by default.

[VAIQ_WARN][QUANTIZER_TORCH_FLOAT_OP]: The quantizer recognize new op `aten::_convolution_mode` as a float operator by default.

[VAIQ_NOTE]: =>Doing weights equalization...

[VAIQ_NOTE]: =>Quantizable module is generated.(inspect/AutoEncoder1D.py)

[VAIQ_WARN]: AutoEncoder1D::3106 is not tensor.

[VAIQ_WARN]: AutoEncoder1D::3115 is not tensor.



[VAIQ_NOTE]: Find subgraph for reshape_fix_1:
node name:AutoEncoder1D::AutoEncoder1D/ConvBlock[downsample_blocks]/ModuleList[1]/ret.57, op type:nndct_reshape, output shape: [4, 64, 64]




I20241109 14:53:56.687400  2735 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20241109 14:53:56.687443  2735 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20241109 14:53:56.687450  2735 compile_pass_manager.cpp:357] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B4096


...


[VAIQ_NOTE]: Find subgraph for concat_fix_13:
node name:AutoEncoder1D::AutoEncoder1D/ret.155, op type:nndct_concat, output shape: [4, 128, 32]




I20241109 14:53:57.138144  2735 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20241109 14:53:57.138166  2735 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20241109 14:53:57.138171  2735 compile_pass_manager.cpp:357] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B4096
I20241109 14:53:57.138196  2735 compile_pass_manager.cpp:465] [UNILOG][INFO] Graph name: nndct_concat_lhsA7pY3kd4IcJqu, with op num: 6
I20241109 14:53:57.138198  2735 compile_pass_manager.cpp:478] [UNILOG][INFO] Begin to compile...
W20241109 14:53:57.139575  2735 PartitionPass.cpp:4160] [UNILOG][WARNING] xir::Op{name = AutoEncoder1D__AutoEncoder1D_ret_199, type = concat-fix} has been assigned to CPU: [it has one input is not from DPU.].
I20241109 14:53:57.139611  2735 compile_pass_manager.cpp:489] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 0
I20241109 14:53:57.139621  2735 compile_pass_manager.cpp:504] [UNILOG][INFO] Compile done.



[VAIQ_NOTE]: The operators assigned to the CPU are as follows(see more details in 'inspect/inspect_DPUCZDX8G_ISA1_B4096.txt'):
node name                                                                                                                                   op Type                  hardware constraints
------------------------------------------------------------------------------------------------------------------------------------------  -----------------------  ------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
AutoEncoder1D::AutoEncoder1D/ConvBlock[spatial_reduce]/ret.5                                                                                nndct_reshape            The input of reshape is not on DPU.
AutoEncoder1D::AutoEncoder1D/ConvBlock[spatial_reduce]/Conv2d[conv1d]/ret.7                                              

I20241109 14:53:57.142123  2735 compile_pass_manager.cpp:352] [UNILOG][INFO] Compile mode: dpu
I20241109 14:53:57.142148  2735 compile_pass_manager.cpp:353] [UNILOG][INFO] Debug mode: null
I20241109 14:53:57.142151  2735 compile_pass_manager.cpp:357] [UNILOG][INFO] Target architecture: DPUCZDX8G_ISA1_B4096
I20241109 14:53:57.142176  2735 compile_pass_manager.cpp:465] [UNILOG][INFO] Graph name: nndct_concat_98jVBcoqi3IMXs1l, with op num: 6
I20241109 14:53:57.142179  2735 compile_pass_manager.cpp:478] [UNILOG][INFO] Begin to compile...
W20241109 14:53:57.143525  2735 PartitionPass.cpp:4160] [UNILOG][WARNING] xir::Op{name = AutoEncoder1D__AutoEncoder1D_ret_155, type = concat-fix} has been assigned to CPU: [it has one input is not from DPU.].
I20241109 14:53:57.143560  2735 compile_pass_manager.cpp:489] [UNILOG][INFO] Total device subgraph number 3, DPU subgraph number 0
I20241109 14:53:57.143569  2735 compile_pass_manager.cpp:504] [UNILOG][INFO] Compile done.



[VAIQ_NOTE]: Dot image is generated.(inspect/inspect_DPUCZDX8G_ISA1_B4096.png)

[VAIQ_NOTE]: =>Finish inspecting.


##### Inspection result anaysis:
1. In this toy model, there are three operations will be assigned to cpu. The reasons for being allocated to the CPU are shown in the column of hardware constraints.
2. The target device of an input node is determined by its consumer.
3. Here, we found that a special permute operation("ToyModel::ToyModel/AdaptiveAvgPool2d[adaptive_avg_pool2d]/146_sink_transpose_0") is inserted by quantizer. Due to data layout difference between Pytorch('NCHW') and XIR('NHWC'), if quantizer inserts some permutes(which the node message in inspect file will inform us about), these permutes may prevent the entire model from being deployed to the target device. Sometimes, we can cancel out this automatically inserted permute by inserting a permute in the original float model, sometimes, we can't.
4. If you want more details about the inspection, see the inspection report under the output directory you specified.
5. The dot image can help you to analyze inspection result in a more intuitive way. "svg" and "png" format are both supported.


In [9]:
model.spatial_reduce

ConvBlock(
  (conv1d): Conv2d(2480, 32, kernel_size=(3, 3), stride=(1, 1), padding=same, bias=False)
  (norm): LayerNorm((32,), eps=1e-05, elementwise_affine=True)
  (activation): GELU(approximate='none')
  (drop): Dropout(p=0.1, inplace=False)
  (downsample): MaxPool2d(kernel_size=1, stride=1, padding=0, dilation=1, ceil_mode=False)
)

In [10]:
# Show the dot image
from IPython.display import Image
# Image('inspect/inspect_DPUCZDX8G_ISA1_B4096.png')

##### How to deploy this entire model on DPU:
1. Find the location of CPU operations in original model. All the details of operations are shown in inspect_{target}.txt, including source range, points to a source which is a stack track and helps to find the exact location of this operation in source code. Take resize op as a example, from source range, we can find the location of resize op in #3 input block line 11.  
node name: ToyModel::ToyModel/Upsample[upsample]/input.3  
...  
op type: resize  
...  
source range:  
...  
<font color=red>ipython-input-3-7ec1fc6b678f(11): forward</font>  
...

  

2. Modify the original model according to the hardware constrains message and try to cancel out the permute inserted by quantizer with by inserting a permute in the original float model(more details see hints in inspect_{target}.txt)